In [1]:
# Let's try to implement a basic RAG system

In [2]:
from dotenv import load_dotenv
load_dotenv() # load environment variables

True

In [3]:
# The system doesn't know anything about Destiny beforehand

from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2:1b")
simple_prompt = PromptTemplate.from_template("{question}")  # pass-through question
simple_chain = simple_prompt | llm

response = simple_chain.invoke({"question": "Who is Destiny on YouTube?"})
print(response)

I couldn't find any information on a person or channel called "Destiny" on YouTube. It's possible that you may have misspelled the name or that Destiny is a relatively new or niche creator. Can you provide more context or details about who or what Destiny is? I'll do my best to help.


In [4]:
# Let's provide the system information about Destiny
#   1. fetch the wikipedia test on Destiny
#   2. split it into chunks
#   3. convert each chunk into a vector embedding
#   4. store everything in a vector database
#
# This is the indexing stage of RAG.

from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.schema import Document
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# fetch (directly)
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Destiny_(streamer)")
docs = loader.load()

# # fetch (via WikipediaAPIWrapper)
# wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# data = wiki.run("Destiny_(streamer)")
# docs = [Document(page_content=data)]

# split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

# embed
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# store (in FAISS)
vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

# # store (in Chroma)
# vectorstore = Chroma.from_documents(
#     documents=documents,
#     embedding=embedding_model,
#     collection_name="docs"
# )

print(documents)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Destiny_(streamer)', 'title': 'Destiny (streamer) - Wikipedia', 'language': 'en'}, page_content='Destiny (streamer) - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools'), Document(metadata={'source': 'https://en.wikipedia.org/wiki/Destiny_(streamer)', 'title': 'Destiny (streamer) - Wikipedia', 'language': 'en'}, page_content='Donate Create account Log in\n\n\n\n\n\n\t\tPages for lo

In [11]:
# Now, let's create a prompt to facilitate RAG:
#  - In our prompt, take a user question (as before) and some *external context*.
#  - Based on the user question, retrieve the most relevant document chunks from the vector store.
#  - Format both the context and question into our prompt and pass it to the LLM.
#
# This is the retrieval stage of RAG.

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

# EXTRA: "If relevant information is not in the context, you may rely on general knowledge."
prompt = PromptTemplate.from_template('''
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Context: {context}
    Question: {question}
    Answer:
''')

qa_chain = (
    {
        "context": vectorstore.as_retriever(),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("Who is Destiny on YouTube?")

'Destiny is an American live streamer and political commentator, also known as "Destiny" or "Steven Kenneth Bonnell II". He was among the first people to stream video games online full-time.'